# Tarea 2 - Introducción a las Redes Neuronales y Deep Learning

**Nombre:** Bruno Morici

**ROL USM:** 202373555-8

**Curso:** INF395, Introducción a las Redes Neuronales y Deep Learning

**Profesor:** Alejandro Veloz

**Fecha:** 21/10/2025

# Parte I: Desbalance de clases

### Pasos:
- Se hará uso de un dataset de clasificación de imágenes
- Artificialmente se limitarán las muestras de una clase para generar un desbalance artificial
- Se creará y entrenará un modelo FANN con las muestras totales
- Luego se harán predicciones midiendo el Acuraccy y el F1-Score (sobre todo en la clase desbalanceada, donde se espera sea cercano a cero)
- Luego se aplicará el método de Data Augmentation, para tomar las pocas muestras de la clase desbalanceada y balancearla a la par de las otras
- Finalmente se reentrenará el modelo, realizando nuevas predicciones y mediciones, para ver el efecto del método de rebalanceamiento.

# Parte II: Sistema de recomendación